<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practice SQL with Pandas pt. 2


---

We've learned about relational databases and the language most use to query them, SQL.  

In this lab we are going to gain more practice converting information to a SQL DB, querying the data and then analyzing it with Python.

In [1]:
# Necessary Libraries
import pandas as pd
import sqlite3
from pandas.io import sql

#### 1.  Read in the EuroMart CSV Data.
- 'EuroMart-ListOfOrders.csv'
- 'EuroMart-OrderBreakdown.csv'
- 'EuroMart-SalesTargets.csv'

In [2]:
# Reading CSV to Dataframe
orders = pd.read_csv('./datasets/csv/EuroMart-ListOfOrders.csv', encoding = 'utf-8')
OBD =  pd.read_csv('./datasets/csv/EuroMart-OrderBreakdown.csv', encoding = 'utf-8')
sales_targets =  pd.read_csv('./datasets/csv/EuroMart-SalesTargets.csv', encoding = 'utf-8')

#### 2. Rename columns to remove any spaces.

In [3]:
# A: 
orders.columns =['Order_ID', 'Order_Date', 'Customer_Name', 'City', 'Country', 'Region',
       'Segment', 'Ship_Date', 'Ship_Mode', 'State']

In [4]:
orders.head()

,Order_ID,Order_Date,Customer_Name,City,Country,Region,Segment,Ship_Date,Ship_Mode,State
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm
1,AZ-2011-9050313,1/3/2011,Summer Hayward,Southport,United Kingdom,North,Consumer,1/7/2011,Economy,England
2,AZ-2011-6674300,1/4/2011,Devin Huddleston,Valence,France,Central,Consumer,1/8/2011,Economy,Auvergne-Rhône-Alpes
3,BN-2011-2819714,1/4/2011,Mary Parker,Birmingham,United Kingdom,North,Corporate,1/9/2011,Economy,England
4,AZ-2011-617423,1/5/2011,Daniel Burke,Echirolles,France,Central,Home Office,1/7/2011,Priority,Auvergne-Rhône-Alpes


In [5]:
OBD.columns

Index(['Order ID', 'Product Name', 'Discount', 'Sales', 'Profit', 'Quantity',
       'Category', 'Sub-Category'],
      dtype='object')

In [6]:
OBD.columns=['Order_ID', 'Product_Name', 'Discount', 'Sales', 'Profit', 'Quantity',
       'Category', 'Sub_Category']

In [7]:
sales_targets.columns

Index(['Month of Order Date', 'Category', 'Target'], dtype='object')

In [8]:
sales_targets.columns= ['Month_of_Order_Date', 'Category', 'Target']

#### 3. Remove dollar signs from sales and profit columns in the order breakdown dataframe.

Convert the columns to float.

In [9]:
# A: 
OBD.head()

,Order_ID,Product_Name,Discount,Sales,Profit,Quantity,Category,Sub_Category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,$45.00,-$26.00,3,Office Supplies,Paper
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,$854.00,$290.00,7,Furniture,Bookcases
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,$140.00,$21.00,3,Office Supplies,Art
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,$27.00,-$22.00,2,Office Supplies,Art
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,$17.00,-$1.00,2,Office Supplies,Storage


In [10]:
OBD['Sales'] = OBD['Sales'].apply(lambda x: float(x.replace('$','').replace(',','')))
OBD['Profit'] = OBD['Profit'].apply(lambda x: float(x.replace('$','').replace(',','')))

#### 4. Create a SQL Database called 'EuroMart' and save the three dataframes as SQL tables. 

In [11]:
# Establishing Local DB connection
db_connection = sqlite3.connect('./datasets/sql/EuroMart.db.sqlite')


In [12]:
# A: 
orders.to_sql(name = 'orders', con = db_connection, if_exists = 'replace', index = False)
OBD.to_sql(name = 'OBD', con = db_connection, if_exists = 'replace', index = False)
sales_targets.to_sql(name = 'sales_target', con = db_connection, if_exists = 'replace', index = False)

#### 5. How many orders has each Customer placed? 

In [32]:
# A:
query = 'select customer_name from orders'

results = sql.read_sql(query, con = db_connection)

In [38]:
results['Customer_Name'].value_counts()

Jose Gambino           13
Mark Washington        12
Kayla Tearle           12
Rachel Tyler           11
Patricia Smith         11
Georgina Garner        11
Lilly Le Grand         11
Aaron Bootman          11
Maya Pamphlett         11
Lori Miller            11
Hayden Perkins         11
Jessica Paramor        11
Michael Myers          11
Jason Roger            11
Terence Welch          11
Angie Massengill       10
Philip Newsom          10
Ngoc Orozco            10
Peter Daly             10
Robert Quick           10
Michael Watkins        10
Alexandra Macdonald    10
Ashton Charles         10
Elijah Sodeman         10
Chelsea Bannister      10
Owen Howell            10
Elise Ali              10
Kian Sykes             10
Isaac David            10
Jasmine Slater         10
                       ..
Grace Brown             1
Elliot Owen             1
Claire Dilke            1
Tegan Savage            1
Shannon Lane            1
Louie Bryant            1
Dakota Baynes           1
Audry Smith 

> *If you're doubting your output check using Pandas*

#### 6. Create a Query to return a table of only geographic features from the List of Orders Table.

In [16]:
# A:
query = 'select City,Country, Region, State from orders'

results = sql.read_sql(query, con = db_connection)

results.head()

,City,Country,Region,State
0,Stockholm,Sweden,North,Stockholm
1,Southport,United Kingdom,North,England
2,Valence,France,Central,Auvergne-Rhône-Alpes
3,Birmingham,United Kingdom,North,England
4,Echirolles,France,Central,Auvergne-Rhône-Alpes


#### 7. Create a Query to return a table with all of the orders that had a negative profit from the Order Breakdown Table.

In [18]:

# A:
query = 'select * from OBD where Profit < 0'

results = sql.read_sql(query, con = db_connection)

results.head()

,Order_ID,Product_Name,Discount,Sales,Profit,Quantity,Category,Sub_Category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
2,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage
3,BN-2011-3248724,"Ikea Classic Bookcase, Metal",0.6,987.0,-1012.0,6,Furniture,Bookcases
4,BN-2011-3248724,"Binney & Smith Sketch Pad, Blue",0.5,116.0,-56.0,5,Office Supplies,Art


#### 8. Construct a query to return a table with the Customer Name and Product Name.  

> **Note:** This will require a join!

In [24]:
orders.head()

,Order_ID,Order_Date,Customer_Name,City,Country,Region,Segment,Ship_Date,Ship_Mode,State
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm
1,AZ-2011-9050313,1/3/2011,Summer Hayward,Southport,United Kingdom,North,Consumer,1/7/2011,Economy,England
2,AZ-2011-6674300,1/4/2011,Devin Huddleston,Valence,France,Central,Consumer,1/8/2011,Economy,Auvergne-Rhône-Alpes
3,BN-2011-2819714,1/4/2011,Mary Parker,Birmingham,United Kingdom,North,Corporate,1/9/2011,Economy,England
4,AZ-2011-617423,1/5/2011,Daniel Burke,Echirolles,France,Central,Home Office,1/7/2011,Priority,Auvergne-Rhône-Alpes


In [25]:
OBD.head()

,Order_ID,Product_Name,Discount,Sales,Profit,Quantity,Category,Sub_Category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,854.0,290.0,7,Furniture,Bookcases
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,21.0,3,Office Supplies,Art
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage


In [20]:

def Q(query, db=db_connection):
    return sql.read_sql(query, db)

In [26]:
# A:
# A:
Q('''
SELECT o.Customer_Name, OBD.Product_Name
FROM orders AS o
INNER JOIN OBD ON o.Order_ID=OBD.Order_ID ''')

,Customer_Name,Product_Name
0,Ruby Patel,"Enermax Note Cards, Premium"
1,Summer Hayward,"Dania Corner Shelving, Traditional"
2,Devin Huddleston,"Binney & Smith Sketch Pad, Easy-Erase"
3,Mary Parker,"Boston Markers, Easy-Erase"
4,Mary Parker,"Eldon Folders, Single Width"
5,Daniel Burke,"Binney & Smith Pencil Sharpener, Water Color"
6,Daniel Burke,"Sanford Canvas, Fluorescent"
7,Fredrick Beveridge,"Accos Thumb Tacks, Assorted Sizes"
8,Fredrick Beveridge,"Bush Floating Shelf Set, Pine"
9,Fredrick Beveridge,"Smead Lockers, Industrial"


#### 9.  How many orders for "Office Supplies" (Category) has Sweden made?

> **Note:** from this point on you'll probably be combining SQL and Pandas, in that you would use SQL querys to gather the relevant information and use Pandas to analyze it.

In [29]:
# A:
sweden_os = Q('''
SELECT *
FROM orders AS o
INNER JOIN OBD ON o.Order_ID=OBD.Order_ID
WHERE OBD.Category="Office Supplies"
AND o.Country = "Sweden"''')

In [31]:
sweden_os.head()

,Order_ID,Order_Date,Customer_Name,City,Country,Region,Segment,Ship_Date,Ship_Mode,State,Order_ID,Product_Name,Discount,Sales,Profit,Quantity,Category,Sub_Category
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,BN-2011-2807470,1/13/2011,Rose Heap,Gothenburg,Sweden,North,Consumer,1/20/2011,Economy,Västra Götaland,BN-2011-2807470,"Sanford Pens, Fluorescent",0.5,31.0,-14.0,5,Office Supplies,Art
2,BN-2011-7087921,2/3/2011,Max Ludwig,Uppsala,Sweden,North,Home Office,2/5/2011,Economy Plus,Uppsala,BN-2011-7087921,"Boston Pencil Sharpener, Easy-Erase",0.5,91.0,-62.0,6,Office Supplies,Art
3,BN-2011-827720,5/9/2011,Ewan Hyde,Stockholm,Sweden,North,Consumer,5/14/2011,Economy,Stockholm,BN-2011-827720,"Sanford Pencil Sharpener, Fluorescent",0.5,26.0,-8.0,2,Office Supplies,Art
4,BN-2011-8234232,5/10/2011,Erick Tanner,Stockholm,Sweden,North,Corporate,5/12/2011,Economy Plus,Stockholm,BN-2011-8234232,"Boston Pencil Sharpener, Water Color",0.5,44.0,-32.0,3,Office Supplies,Art


In [30]:
len(sweden_os)

133

#### 10. What was the total sales for products that have been discounted? 

In [40]:
# A:
discounted_sales = Q('''
SELECT *
FROM OBD
WHERE discount>0
''')

In [41]:
discounted_sales.head()

,Order_ID,Product_Name,Discount,Sales,Profit,Quantity,Category,Sub_Category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
2,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage
3,AZ-2011-2918397,"Bush Floating Shelf Set, Pine",0.1,155.0,36.0,1,Furniture,Bookcases
4,AZ-2011-2918397,"Smead Lockers, Industrial",0.1,716.0,143.0,4,Office Supplies,Storage


In [42]:
discounted_sales['Sales'].sum()

1115614.0

#### 11. What is the total quantity of objects sold for each country?

In [45]:
orders.head()

,Order_ID,Order_Date,Customer_Name,City,Country,Region,Segment,Ship_Date,Ship_Mode,State
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm
1,AZ-2011-9050313,1/3/2011,Summer Hayward,Southport,United Kingdom,North,Consumer,1/7/2011,Economy,England
2,AZ-2011-6674300,1/4/2011,Devin Huddleston,Valence,France,Central,Consumer,1/8/2011,Economy,Auvergne-Rhône-Alpes
3,BN-2011-2819714,1/4/2011,Mary Parker,Birmingham,United Kingdom,North,Corporate,1/9/2011,Economy,England
4,AZ-2011-617423,1/5/2011,Daniel Burke,Echirolles,France,Central,Home Office,1/7/2011,Priority,Auvergne-Rhône-Alpes


In [47]:
# A:
Q('''
SELECT o.country, SUM(OBD.Quantity)
FROM orders AS o
INNER JOIN OBD ON o.Order_ID=OBD.Order_ID
GROUP BY o.country''')

,Country,SUM(OBD.Quantity)
0,Austria,973
1,Belgium,532
2,Denmark,204
3,Finland,201
4,France,7329
5,Germany,6179
6,Ireland,392
7,Italy,3612
8,Netherlands,1526
9,Norway,261


#### 12. In what Countries are profits lowest? (Report lowest 5-10)

In [49]:
# A:
Q('''
SELECT o.country, SUM(OBD.Profit)
FROM orders AS o
INNER JOIN OBD ON o.Order_ID=OBD.Order_ID
GROUP BY o.country
ORDER BY SUM(OBD.Profit) 
LIMIT 5''')

,Country,SUM(OBD.Profit)
0,Netherlands,-37188.0
1,Sweden,-17524.0
2,Portugal,-8704.0
3,Ireland,-6886.0
4,Denmark,-3608.0


#### 13. What Counties have the best and worst Sales to Profit Ratios?
(Total Sales divided by Total Profits.)
Essentially this is saying for every dollar of product sold, how much is profit.

In [52]:
# A:
df = Q('''
SELECT o.country, OBD.Profit, OBD.Sales
FROM orders AS o
INNER JOIN OBD ON o.Order_ID=OBD.Order_ID
''')

In [57]:
new_df = df.groupby('Country').sum()
new_df.head()

,Profit,Sales
Country,,
Austria,21332.0,79382.0
Belgium,9912.0,42320.0
Denmark,-3608.0,7763.0
Finland,3908.0,20702.0
France,70067.0,609683.0


In [58]:
new_df['ratio'] = new_df['Profit']/new_df['Sales']

In [60]:
new_df.sort_values('ratio')

,Profit,Sales,ratio
Country,,,
Portugal,-8704.0,15106.0,-0.576195
Sweden,-17524.0,30490.0,-0.574746
Netherlands,-37188.0,70313.0,-0.528892
Denmark,-3608.0,7763.0,-0.464769
Ireland,-6886.0,15998.0,-0.430429
Italy,15802.0,252742.0,0.062522
France,70067.0,609683.0,0.114924
Germany,86279.0,488681.0,0.176555
Spain,47067.0,249402.0,0.188719


#### 14. What Shipping method is most common for 'Bookcases' (Sub Category)?

In [68]:
# A:

df = Q('''
SELECT o.Ship_mode, OBD.Sub_Category
FROM orders AS o
INNER JOIN OBD ON o.Order_ID=OBD.Order_ID
WHERE OBD.Sub_Category ="Bookcases"
''')

In [69]:
df.head()

,Ship_Mode,Sub_Category
0,Economy,Bookcases
1,Priority,Bookcases
2,Economy,Bookcases
3,Economy,Bookcases
4,Economy,Bookcases


In [71]:
df['Ship_Mode'].value_counts()

Economy         234
Economy Plus     76
Priority         59
Immediate        22
Name: Ship_Mode, dtype: int64

#### 15 .What city in the Orders table generated the highest net sales?  (List all the cities and countries in descending order by net sales.)

In [74]:
# A:

df = Q('''
SELECT o.city, o.country, OBD.Sales, OBD.profit
FROM orders AS o
INNER JOIN OBD ON o.Order_ID=OBD.Order_ID''')

In [75]:
df.head()

,City,Country,Sales,Profit
0,Stockholm,Sweden,45.0,-26.0
1,Southport,United Kingdom,854.0,290.0
2,Valence,France,140.0,21.0
3,Birmingham,United Kingdom,17.0,-1.0
4,Birmingham,United Kingdom,27.0,-22.0


In [78]:
new_df = df.groupby(['City','Country']).sum()

In [79]:
new_df.head()

,,Sales,Profit
City,Country,,
Aachen,Germany,1419.0,372.0
Aalen,Germany,980.0,283.0
Aalst,Belgium,388.0,152.0
Abbeville,France,218.0,10.0
Aberdeen,United Kingdom,1547.0,525.0


In [80]:
new_df['net_sales']=new_df['Sales']-new_df['Profit']
new_df.head()

,,Sales,Profit,net_sales
City,Country,,,
Aachen,Germany,1419.0,372.0,1047.0
Aalen,Germany,980.0,283.0,697.0
Aalst,Belgium,388.0,152.0,236.0
Abbeville,France,218.0,10.0,208.0
Aberdeen,United Kingdom,1547.0,525.0,1022.0


In [81]:
new_df.sort_values('net_sales')

,,Sales,Profit,net_sales
City,Country,,,
Osny,France,5.0,2.0,3.0
Aix-les-Bains,France,10.0,4.0,6.0
Sète,France,14.0,7.0,7.0
Sorgues,France,14.0,3.0,11.0
Friedberg,Germany,18.0,6.0,12.0
Cluses,France,28.0,14.0,14.0
Ragusa,Italy,22.0,7.0,15.0
Viroflay,France,26.0,11.0,15.0
Givors,France,26.0,10.0,16.0


#### BONUS: Create a Column called 'Shipping Delay' on the 'orders' table, which is the difference in days between 'Order Date' and 'Ship Date'.

In [18]:
# A:

In [19]:
# A:

#### BONUS: Update your Orders table in your Sqlite DB to include the 'Shipping Delay' feature.

In [20]:
# A:

#### BONUS: Which Product Category has the highest average 'Shipping Delay'?

In [21]:
# A:

### Challenge problem:   
**In what months and Categories were Sales Targets Exceeded?**

---

This may require a considerable amount of data processing.

In [22]:
# A: